In [1]:
import os
import torch
import numpy as np
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from tqdm import tqdm

# Paths
data_dir = "normalized_images/efficientnet"
output_feature_path = "efficientnet_features.npy"
output_label_path = "efficientnet_labels.npy"

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom Dataset
class EfficientNetImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_names = sorted(os.listdir(root_dir))
        for label_index, class_name in enumerate(self.class_names):
            class_dir = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                self.image_paths.append(os.path.join(class_dir, img_name))
                self.labels.append(label_index)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]

# Transformation (no resizing or normalization needed)
transform = transforms.ToTensor()

# Dataset and DataLoader
dataset = EfficientNetImageDataset(data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

# Load EfficientNet-B0 without classification head
model = models.efficientnet_b0(pretrained=True)
model.classifier = torch.nn.Identity()
model = model.to(device)
model.eval()

# Extract features
features_list = []
labels_list = []

with torch.no_grad():
    for images, labels in tqdm(dataloader, desc="Extracting EfficientNet features"):
        images = images.to(device)
        outputs = model(images)
        features_list.append(outputs.cpu().numpy())
        labels_list.extend(labels.numpy())

# Save to disk
features_array = np.vstack(features_list)
labels_array = np.array(labels_list)

np.save(output_feature_path, features_array)
np.save(output_label_path, labels_array)

print("Features saved to:", output_feature_path)
print("Labels saved to:", output_label_path)


D:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Extracting EfficientNet features: 100%|██████████| 219/219 [03:15<00:00,  1.12it/s]

Features saved to: efficientnet_features.npy
Labels saved to: efficientnet_labels.npy


In [3]:
import os
import torch
import numpy as np
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from tqdm import tqdm

# Paths
data_dir = "normalized_images/vgg"
output_feature_path = "vgg_features.npy"
output_label_path = "vgg_labels.npy"

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom Dataset
class VGGImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_names = sorted(os.listdir(root_dir))
        for label_index, class_name in enumerate(self.class_names):
            class_dir = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                self.image_paths.append(os.path.join(class_dir, img_name))
                self.labels.append(label_index)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]

# Transformation (we assume normalization already done)
transform = transforms.ToTensor()

# Dataset and DataLoader
dataset = VGGImageDataset(data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

# Load VGG16 and remove the classifier (use only feature extractor)
vgg = models.vgg16(pretrained=True)
vgg.classifier = torch.nn.Identity()
vgg = vgg.to(device)
vgg.eval()

# Extract features
features_list = []
labels_list = []

with torch.no_grad():
    for images, labels in tqdm(dataloader, desc="Extracting VGG16 features"):
        images = images.to(device)
        outputs = vgg(images)
        features_list.append(outputs.cpu().numpy())
        labels_list.extend(labels.numpy())

# Save features and labels
features_array = np.vstack(features_list)
labels_array = np.array(labels_list)

np.save(output_feature_path, features_array)
np.save(output_label_path, labels_array)

print("VGG features saved to:", output_feature_path)
print("Labels saved to:", output_label_path)


D:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Extracting VGG16 features: 100%|██████████| 219/219 [13:02<00:00,  3.57s/it]


VGG features saved to: vgg_features.npy
Labels saved to: vgg_labels.npy


In [5]:
import os
import torch
import numpy as np
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from tqdm import tqdm

# Paths
data_dir = "normalized_images/resnet"
output_feature_path = "resnet_features.npy"
output_label_path = "resnet_labels.npy"

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom Dataset
class ResNetImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_names = sorted(os.listdir(root_dir))
        for label_index, class_name in enumerate(self.class_names):
            class_dir = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                self.image_paths.append(os.path.join(class_dir, img_name))
                self.labels.append(label_index)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]

# Transformation
transform = transforms.ToTensor()

# Dataset and DataLoader
dataset = ResNetImageDataset(data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

# Load ResNet50 and remove the final FC layer
resnet = models.resnet50(pretrained=True)
resnet.fc = torch.nn.Identity()  # Remove classification layer
resnet = resnet.to(device)
resnet.eval()

# Extract features
features_list = []
labels_list = []

with torch.no_grad():
    for images, labels in tqdm(dataloader, desc="Extracting ResNet50 features"):
        images = images.to(device)
        outputs = resnet(images)
        features_list.append(outputs.cpu().numpy())
        labels_list.extend(labels.numpy())

# Save features and labels
features_array = np.vstack(features_list)
labels_array = np.array(labels_list)

np.save(output_feature_path, features_array)
np.save(output_label_path, labels_array)

print("ResNet50 features saved to:", output_feature_path)
print("Labels saved to:", output_label_path)


D:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Extracting ResNet50 features: 100%|██████████| 219/219 [05:55<00:00,  1.62s/it]

ResNet50 features saved to: resnet_features.npy
Labels saved to: resnet_labels.npy


In [7]:
import os
import torch
import numpy as np
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from tqdm import tqdm

# Paths
data_dir = "normalized_images/densenet"
output_feature_path = "densenet_features.npy"
output_label_path = "densenet_labels.npy"

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom Dataset
class DenseNetImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_names = sorted(os.listdir(root_dir))
        for label_index, class_name in enumerate(self.class_names):
            class_dir = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                self.image_paths.append(os.path.join(class_dir, img_name))
                self.labels.append(label_index)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]

# Transformation
transform = transforms.ToTensor()

# Dataset and DataLoader
dataset = DenseNetImageDataset(data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

# Load DenseNet121 and remove classifier
densenet = models.densenet121(pretrained=True)
densenet.classifier = torch.nn.Identity()  # Remove final classification layer
densenet = densenet.to(device)
densenet.eval()

# Extract features
features_list = []
labels_list = []

with torch.no_grad():
    for images, labels in tqdm(dataloader, desc="Extracting DenseNet121 features"):
        images = images.to(device)
        outputs = densenet(images)
        features_list.append(outputs.cpu().numpy())
        labels_list.extend(labels.numpy())

# Save features and labels
features_array = np.vstack(features_list)
labels_array = np.array(labels_list)

np.save(output_feature_path, features_array)
np.save(output_label_path, labels_array)

print("DenseNet121 features saved to:", output_feature_path)
print("Labels saved to:", output_label_path)

D:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Extracting DenseNet121 features: 100%|██████████| 219/219 [05:49<00:00,  1.59s/it]

DenseNet121 features saved to: densenet_features.npy
Labels saved to: densenet_labels.npy


In [11]:
import os
import torch
import numpy as np
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from tqdm import tqdm

# Paths
data_dir = "normalized_images/mobilenetv2"
output_feature_path = "mobilenet_features.npy"
output_label_path = "mobilenet_labels.npy"

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom Dataset
class MobileNetImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_names = sorted(os.listdir(root_dir))
        for label_index, class_name in enumerate(self.class_names):
            class_dir = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                self.image_paths.append(os.path.join(class_dir, img_name))
                self.labels.append(label_index)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]

# Transformation
transform = transforms.ToTensor()

# Dataset and DataLoader
dataset = MobileNetImageDataset(data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

# Load MobileNetV2 and remove classifier
mobilenet = models.mobilenet_v2(pretrained=True)
mobilenet.classifier = torch.nn.Identity()  # Remove classification layer
mobilenet = mobilenet.to(device)
mobilenet.eval()

# Extract features
features_list = []
labels_list = []

with torch.no_grad():
    for images, labels in tqdm(dataloader, desc="Extracting MobileNetV2 features"):
        images = images.to(device)
        outputs = mobilenet(images)
        features_list.append(outputs.cpu().numpy())
        labels_list.extend(labels.numpy())

# Save features and labels
features_array = np.vstack(features_list)
labels_array = np.array(labels_list)

np.save(output_feature_path, features_array)
np.save(output_label_path, labels_array)

print("MobileNetV2 features saved to:", output_feature_path)
print("Labels saved to:", output_label_path)

D:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Extracting MobileNetV2 features: 100%|██████████| 219/219 [02:32<00:00,  1.44it/s]

MobileNetV2 features saved to: mobilenet_features.npy
Labels saved to: mobilenet_labels.npy
